This file is for exploring how the data is structured in the given files

In [26]:
import pandas as pd
import os

Set the working dir and define paths

In [27]:
notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)
input_dir = os.path.join(parent_dir, 'raw', 'unzipped')
output_path = os.path.join(parent_dir, 'joined', 'operational_data.csv')
sensor_dictionary_path = os.path.join(parent_dir, 'metadata', 'Metrics registration.csv')

Example for one file

In [28]:
def load_single_data_file(file_path, header, sensor_dict_df):

    # load df from file path
    df = pd.read_csv(file_path, names=header)
    print(f'shape of df before merge: {df.shape}')

    # Add the two columns from the sensor dictionary df "quantity_name" and "source_name" by left joining on "qid_mapping" and "sensor_id"
    df = df.merge(sensor_dict_df[['qid_mapping', 'quantity_name', 'source_name', 'unit']], left_on='qid_mapping', right_on='qid_mapping', how='left')
    print(f'shape of df after merge: {df.shape}')

    # convert the column 'utc_timestamp' to datetime
    df['utc_timestamp'] = pd.to_datetime(df['utc_timestamp'])

    # Sort the rows by the UTC timestamp
    df = df.sort_values(by='utc_timestamp').reset_index(drop=True)

    return df

# initialize an empty dataframe with the headers as columns

def combine_data_files(input_dir, header, sensor_dict_df):

    df_combined = pd.DataFrame(columns=header + ['quantity_name', 'source_name', 'unit'])

    for i in range(1, 13):
        file_path = os.path.join(input_dir, str(i), f'{i}.csv')
        df = load_single_data_file(file_path, header, sensor_dict_df)
        if list(df.columns) == list(df_combined.columns):
            df_combined = pd.concat([df_combined, df], ignore_index=True)
            print(f'Appended data from file: {file_path}. shape of combined df: {df_combined.shape}')
        else:
            print(f'Column names do not match for file: {file_path}. Stopped merging.')
            print(f'columns in combined df: {df_combined.columns}')
            print(f'columns in current df: {df.columns}')
            raise ValueError('Column names do not match.')
        
    # move "value" to the rightmost column
    value_col = df_combined.pop('value')
    df_combined['value'] = value_col

    # Make a new column that does the following to each row: find the newest measurement from the same qid_mapping and calculate the time between them
    df_combined['time_since_last_measurement'] = df_combined.groupby('qid_mapping')['utc_timestamp'].transform(lambda x: x.max() - x)

    return df_combined

In [29]:
header = ['utc_timestamp', 'qid_mapping', 'value']
sensor_dict_df = pd.read_csv(sensor_dictionary_path)

Appending single files into one long dataframe with all observations

In [30]:
df_combined = combine_data_files(input_dir, header, sensor_dict_df)

shape of df before merge: (3657054, 3)
shape of df after merge: (3657054, 6)


C:\Users\August Bjerg-Heise\AppData\Local\Temp\ipykernel_4284\4182191771.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_combined, df], ignore_index=True)


Appended data from file: c:\Users\August Bjerg-Heise\OneDrive - CBS - Copenhagen Business School\Desktop\MSc Business Admin & Data Science\Master's Thesis\05 Repo\Master's thesis\code\data\raw\unzipped\1\1.csv. shape of combined df: (3657054, 6)
shape of df before merge: (3510816, 3)
shape of df after merge: (3510816, 6)
Appended data from file: c:\Users\August Bjerg-Heise\OneDrive - CBS - Copenhagen Business School\Desktop\MSc Business Admin & Data Science\Master's Thesis\05 Repo\Master's thesis\code\data\raw\unzipped\2\2.csv. shape of combined df: (7167870, 6)
shape of df before merge: (3368554, 3)
shape of df after merge: (3368554, 6)
Appended data from file: c:\Users\August Bjerg-Heise\OneDrive - CBS - Copenhagen Business School\Desktop\MSc Business Admin & Data Science\Master's Thesis\05 Repo\Master's thesis\code\data\raw\unzipped\3\3.csv. shape of combined df: (10536424, 6)
shape of df before merge: (3517849, 3)
shape of df after merge: (3517849, 6)
Appended data from file: c:\Us

In [31]:
df_combined.columns.to_list()

['utc_timestamp',
 'qid_mapping',
 'quantity_name',
 'source_name',
 'unit',
 'value',
 'time_since_last_measurement']

Exploration

In [32]:
# Make a summary table with a nested structure that calculates the timespan between the first and last measurement of each sensor. The rows should be grouped by: 1. the "source_name", then 2. "quantity_name", and finally 3. "qid_mapping"
def summarize_sensor_indicators(combined_df):
    summary = combined_df.groupby(['source_name', 'quantity_name', 'qid_mapping']).agg(
        total_measurements=('utc_timestamp', 'count'),
        timespan_days=('utc_timestamp', lambda x: (x.max() - x.min()).days),
        min_value=('value', 'min'),
        max_value=('value', 'max'),
        mean_value=('value', 'mean'),
        std_value=('value', 'std'),
        avg_time_between_measurements=('time_since_last_measurement', lambda x: x.mean().total_seconds()),  # in seconds
        most_frequent_time_between_measurements=('time_since_last_measurement', lambda x: x.mode()[0].total_seconds())  # in seconds
    )

    # add a unit column by looking up the unit in the qid_mapping from the sensor dictionary
    sensor_dict_df = pd.read_csv(sensor_dictionary_path)
    summary['unit'] = summary.index.get_level_values('qid_mapping').map(
        sensor_dict_df.set_index('qid_mapping')['unit']
    )

    # Drop the qid_mapping column for aesthetics
    summary = summary.reset_index(level='qid_mapping', drop=True)

    return summary

In [ ]:
sensor_summary = summarize_sensor_indicators(df_combined)
sensor_summary.reset_index(inplace=True)

pd.set_option('display.float_format', '{:.7f}'.format)

sensor_summary

# TODO: Explain the following:
# Apparent zero-measurements on the draft sensors?
# Lack of unit in vessel propeller shaft revolutions 
# why most common distance in measurements is always 0? (check descriptive statistics for that column further up, probably a problem with that formula)
# Make date-time values in summary more readable (e.g. make response in a date-time format rather than float num of seconds)

,source_name,quantity_name,total_measurements,timespan_days,min_value,max_value,mean_value,std_value,avg_time_between_measurements,most_frequent_time_between_measurements,unit
0,Control Alarm Monitoring System,Main Engine Rotational Speed,1882493,365,-68.3000000,108.9000000,64.1438928,35.8814737,16083060.1098330,0.0000000,rpm
1,Control Alarm Monitoring System,Vessel Hull Aft Draft,1882487,365,0.0000000,0.0000000,0.0000000,0.0000000,16083007.6087540,0.0000000,m
2,Control Alarm Monitoring System,Vessel Hull Fore Draft,1882876,365,0.0000000,0.0000000,0.0000000,0.0000000,16083154.8835850,0.0000000,m
3,Control Alarm Monitoring System,Vessel Hull MidP Draft,1882871,365,0.0000000,0.0000000,0.0000000,0.0000000,16083135.6927500,0.0000000,m
4,Control Alarm Monitoring System,Vessel Hull MidS Draft,1882884,365,0.0000000,0.0000000,0.0000000,0.0000000,16083195.7239580,0.0000000,m
5,Instrument Anemometer,Vessel External Conditions Wind Relative Angle,1977274,365,0.0000000,359.0000000,204.0812710,138.4688858,16269474.7863660,0.0000000,degrees
6,Instrument Anemometer,Vessel External Conditions Wind Relative Speed,1975954,365,0.0000000,70.4000000,19.1619881,10.8648863,16272172.8059780,0.0000000,knots
7,Instrument Echosounder,Vessel Hull Relative To Transducer Water Depth,586021,365,0.7000000,830.6000000,23.5403003,42.2212756,17255546.4216520,0.0000000,m
8,Instrument GPS 1,Vessel Hull Over Ground Speed,1982354,365,0.0000000,15.3000000,8.5319035,4.8039247,16277513.6291310,0.0000000,knots
9,Instrument Gyrocompass,Vessel Hull Heading True Angle,1981852,365,0.0000000,359.9000000,164.6850309,102.8133574,16274917.7086810,0.0000000,degrees
